# Naive Bayes Classification of the comments

# Installing necessary libraries

Pass this step this if the libries are already installed.

In [2]:
!npm install pandas
!npm install numpy
%pip install seaborn
!npm install matplotlib
%pip install scikit-learn


up to date, audited 2 packages in 2s

found 0 vulnerabilities

added 1 package, and audited 3 packages in 2s

found 0 vulnerabilities
Note: you may need to restart the kernel to use updated packages.

added 1 package, and audited 4 packages in 1s

found 0 vulnerabilities
^C
Note: you may need to restart the kernel to use updated packages.


# Importing neccessary libraries

Importing all neccessary libraries that will be used in this Jupyter notebook.

In [32]:
import os
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

# Preparing the collected comments from the vader script

Reading the collected comments from the .csv file.

In [67]:
df_bad_comments = pd.read_csv('bad_comments.csv')
df_good_comments = pd.read_csv('good_comments.csv')

In [68]:
df_bad_comments.head()

,file,comment,sentiment
0,reviews_140607.json,**The Force Awakens: A New Hope 2.0**\r\n\r\nT...,bad
1,reviews_140607.json,A meaningless continuation of a saga that was ...,bad
2,reviews_141052.json,An enjoyable superhero team movie. It's really...,bad
3,reviews_141052.json,Never lived up to its hype. The story was a bi...,bad
4,reviews_141052.json,The biggest flaw of the film was that it didn’...,bad


In [69]:
df_good_comments.head()

,file,comment,sentiment
0,reviews_118340.json,This movie was so AWESOME! I loved it all and ...,good
1,reviews_118340.json,Really funny and with great spcial effects. Pr...,good
2,reviews_118340.json,"This was a pleasant surprise of a movie, full ...",good
3,reviews_118340.json,"Like most of the reviews here, I agree that Gu...",good
4,reviews_118340.json,"The thing about this film when I first saw it,...",good


Adding a new column by name 'class' to each DataFrame to indicate the sentiment.

In [70]:
df_bad_comments['class'] = 0
df_good_comments['class'] = 1

In [71]:
df_bad_comments.shape, df_good_comments.shape

((50, 4), (214, 4))

In [72]:
df_bad_comments_test = df_bad_comments.tail(10)
for i in range(49, 39, -1):
    df_bad_comments.drop([i], axis=0, inplace=True)

df_good_comments_test = df_good_comments.tail(10)
for i in range(213, 193, -1):
    df_good_comments.drop([i], axis=0, inplace=True)

In [73]:
df_bad_comments_test['class'] = 0
df_good_comments_test['class'] = 1

C:\Users\Rebeka\AppData\Local\Temp\ipykernel_11888\3254602890.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bad_comments_test['class'] = 0
C:\Users\Rebeka\AppData\Local\Temp\ipykernel_11888\3254602890.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_good_comments_test['class'] = 1


In [74]:
df_bad_comments_test.head(5)

,file,comment,sentiment,class
40,reviews_569094.json,Incredible non stop action movie,bad,0
41,reviews_572802.json,"So here's good old ""Arthur"" (Jason Momoa) sitt...",bad,0
42,reviews_572802.json,This movie was really quite terrible. I didn't...,bad,0
43,reviews_575264.json,Spectacular cinematic action let down by an in...,bad,0
44,reviews_575264.json,"This is the weakest M:I movie so far, which do...",bad,0


In [75]:
df_good_comments_test.head(5)

,file,comment,sentiment,class
204,reviews_791373.json,_Zack Snyder's Justice League_ is able to turn...,good,1
205,reviews_791373.json,Yes it IS better than the previous version. Th...,good,1
206,reviews_791373.json,**So long but so incredible! DC Comics fans re...,good,1
207,reviews_791373.json,Some HBO Max subscribers were accidentally abl...,good,1
208,reviews_912649.json,A solid enough send off for this <em>'Venom'</...,good,1


In [76]:
df_manual_test = pd.concat([df_bad_comments_test, df_good_comments_test], axis=0)
df_manual_test.to_csv('manual_testing.csv')

Merging of the goood and the bad comments.

In [43]:
df_merged_comments = pd.concat([df_bad_comments, df_good_comments], axis=0)
df_merged_comments.head()

,file,comment,sentiment,class
0,reviews_140607.json,**The Force Awakens: A New Hope 2.0**\r\n\r\nT...,bad,0
1,reviews_140607.json,A meaningless continuation of a saga that was ...,bad,0
2,reviews_141052.json,An enjoyable superhero team movie. It's really...,bad,0
3,reviews_141052.json,Never lived up to its hype. The story was a bi...,bad,0
4,reviews_141052.json,The biggest flaw of the film was that it didn’...,bad,0


In [12]:
df_merged_comments.columns

Index(['file', 'comment', 'sentiment', 'class'], dtype='object')

In [13]:
df_comments = df_merged_comments.drop(['file','sentiment'], axis = 1)

In [14]:
df_comments.isnull().sum()

comment    0
class      0
dtype: int64

Fraction function was used here for the random shuffling of the data frame.

In [27]:
df_comments = df_comments.sample(frac=1)
df_comments.head()

,comment,class
7164,"Netru Indru begins with the killing of Veera, ...",1
2462,"I suppose a sequel was inevitable, but I can't...",0
3927,Terrible talk animal film just like the other ...,0
1476,This was the first movie I ever seen with Will...,1
1383,An odd thing.\r\n\r\nFollowing 10 years on fro...,0


Replacing the current index with a new sequential integer index and adds the old index as a column named 'index'. \
After the column 'index' is removed only index values are left.

In [29]:
df_comments.reset_index(inplace=True)
df_comments.drop(['index'], axis=1, inplace=True)

In [30]:
df_comments.columns

Index(['comment', 'class'], dtype='object')

In [34]:
df_comments.head()

,comment,class
0,"Netru Indru begins with the killing of Veera, ...",1
1,"I suppose a sequel was inevitable, but I can't...",0
2,Terrible talk animal film just like the other ...,0
3,This was the first movie I ever seen with Will...,1
4,An odd thing.\r\n\r\nFollowing 10 years on fro...,0


Pre-processing the comments. 

In [35]:
def preprocess_comments(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\[.*?\]', '', text)  # Remove text within square brackets
    text = re.sub(r'\W', ' ', text)  # Remove non-word characters 
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text).strip()  # Remove URLs
    text = re.sub(r'<.*?>+', '', text)  # Remove HTML tags
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)  # Remove punctuation
    text = re.sub(r'\n', ' ', text).strip()  # Remove newlines
    text = re.sub(r'\r', '', text).strip()  # Remove carriage returns
    text = re.sub(r'\w*\d\w*', '', text)  # Remove words containing digits
    return text


In [37]:
df_comments['comment'] = df_comments ['comment'].apply(preprocess_comments)

In [45]:
X = df_comments['comment']
y = df_comments['class']

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

## Logistic Regression Classifier

In [51]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train_vectorized, y_train)

LogisticRegression()

In [52]:
pred_lr = LR.predict(X_test_vectorized)
LR.score(X_test_vectorized, y_test)

0.8483290488431876

In [53]:
print(classification_report(y_test, pred_lr, zero_division=0))

              precision    recall  f1-score   support

           0       0.85      0.34      0.49      1235
           1       0.85      0.98      0.91      4600

    accuracy                           0.85      5835
   macro avg       0.85      0.66      0.70      5835
weighted avg       0.85      0.85      0.82      5835



## Naive Bayes Classifier

In [58]:
from sklearn.naive_bayes import BernoulliNB
NB = BernoulliNB()
NB.fit(X_train_vectorized, y_train)

BernoulliNB()

In [59]:
pred_nb = NB.predict(X_test_vectorized)
NB.score(X_test_vectorized, y_test)

0.8006855184233076

In [60]:
print("                 Naive Bayes Classification Report:")
print(classification_report(y_test, pred_nb, zero_division=0))

                 Naive Bayes Classification Report:
              precision    recall  f1-score   support

           0       0.60      0.18      0.27      1235
           1       0.81      0.97      0.88      4600

    accuracy                           0.80      5835
   macro avg       0.71      0.57      0.58      5835
weighted avg       0.77      0.80      0.76      5835



## Random Forest Classifier

In [61]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(random_state=0)
RF.fit(X_train_vectorized, y_train)

RandomForestClassifier(random_state=0)

In [62]:
pred_rf = RF.predict(X_test_vectorized)

In [63]:
RF.score(X_test_vectorized, y_test)

0.8053127677806341

In [64]:
print("                Random Forest Classification Report:")
print(classification_report(y_test, pred_rf, zero_division=0))

                Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.09      0.17      1235
           1       0.80      1.00      0.89      4600

    accuracy                           0.81      5835
   macro avg       0.84      0.54      0.53      5835
weighted avg       0.82      0.81      0.74      5835



## Decisiont Tree Classification

In [65]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier(random_state=42)
DT.fit(X_train_vectorized, y_train)

DecisionTreeClassifier(random_state=42)

In [66]:
pred_dt = DT.predict(X_test_vectorized)

In [67]:

DT.score(X_test_vectorized, y_test)

0.7513281919451585

In [68]:
print("                Decision Tree Classification Report:")
print(classification_report(y_test, pred_dt, zero_division=0))

                Decision Tree Classification Report:
              precision    recall  f1-score   support

           0       0.41      0.39      0.40      1235
           1       0.84      0.85      0.84      4600

    accuracy                           0.75      5835
   macro avg       0.62      0.62      0.62      5835
weighted avg       0.75      0.75      0.75      5835



## Testiranje

Running new comments through Logistic Regression, Naive Bayes, Random Forest and Decision Tree classifiers trained models.

In [ ]:
def output_label(n):
    if n == 0:
        return 'Bad comment'
    elif n == 1:
        return 'Good comment'
    
def manual_testing(comment):
    testing_comments = {"comment": [comment]}
    new_def_test = pd.DataFrame(testing_comments)
    new_def_test['comment'] = new_def_test['comment'].apply(preprocess_comments)
    new_x_test = new_def_test['comment']
    new_x_test_vectorized = vectorizer.transform(new_x_test)
    pred_lr = LR.predict(new_x_test_vectorized)
    pred_nb = NB.predict(new_x_test_vectorized)
    pred_rf = RF.predict(new_x_test_vectorized)
    pred_dt = DT.predict(new_x_test_vectorized)

    return print(f" Logistic Regression Prediction: {output_label(pred_lr)}\n Naive Bayes Prediction: {output_label(pred_nb)}\n \
                 Random Forest Prediction: {output_label(pred_rf)} \n Decision Tree Prediction: {output_label(pred_dt)}")     

In [77]:
test_comment = str(input('Enter a comment to test: '))
manual_testing(test_comment)

 Logistic Regression Prediction: Bad comment
 Naive Bayes Prediction: Good comment
 Random Forest Prediction: Good comment 
 Decision Tree Prediction: Bad comment
